In [1]:
import os
import re
import json
import tempfile
import folium
import ipysheet
import numpy as np
import pandas as pd
import rasterio as rio
import rioxarray as rxr
import geopandas as gpd
import branca.colormap as cm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import ipywidgets as widgets

from glob import glob
from pathlib import Path
from folium import plugins
from pyproj import Transformer
from ipyfilechooser import FileChooser
from ipysheet import from_dataframe
from rasterio.warp import calculate_default_transform, reproject, Resampling
from localtileserver import TileClient, get_leaflet_tile_layer, examples
from ipyleaflet import Map, Marker, basemaps, ScaleControl, LayersControl, AwesomeIcon
from ipyleaflet import LegendControl, FullScreenControl, MarkerCluster

os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = \
    f"{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}"

from localtileserver import TileClient, get_leaflet_tile_layer
from localtileserver import examples, helpers
from ipyleaflet import Map, SplitMapControl
import rasterio as rio

In [2]:
# path = "/explore/nobackup/projects/ilab/projects/AIML_CHM/CNN_CHM/products/senegal/v2/Tappan"
# filename = "Tappan01_WV02_20110430_M1BS_103001000A27E100_data.tif"
path, filename = os.path.split('/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_2/WV03_20190401_M1BS_10400100495FC500-sr-02m.tif')
data_bands = [5, 7, 2]

In [3]:
path

'/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_2'

In [4]:
client = TileClient(os.path.join(path, filename))

sigma = 2
style_list = []
for bid, pid in zip(data_bands, ['#f00', '#0f0', '#00f']):
    band_stats = client.rasterio.statistics(bid)
    newmin = band_stats.mean - (band_stats.std * sigma)
    newmax = band_stats.mean + (band_stats.std * sigma)
    style_list.append(
        {'band': bid, 'palette': pid, 'min': newmin, 'max': newmax})

rgb = get_leaflet_tile_layer(
        client, show=False, band=data_bands,
        scheme='linear', dtype='uint16', style={'bands': style_list}
)

toa = get_leaflet_tile_layer(client)

m = Map(
    center=client.center(), zoom=client.default_zoom, basemap=basemaps.OpenTopoMap, scroll_wheel_zoom=True)
m.add_layer(toa)
m

Map(center=[54.77544560145951, -131.48034064478975], controls=(ZoomControl(options=['position', 'zoom_in_text'…